<a href="https://colab.research.google.com/github/masunomatiko/lab/blob/master/Python%E3%81%8B%E3%82%89DBpedia%E3%81%AESPARQL_endpoint%E3%82%92%E5%8F%A9%E3%81%8F.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## やりたいこと
*   DBpediaから日付型のプロパティを取得する
*   特定のプロパティをもつページのID `wikiPageID` とその述語 `value` を取得する


## ライブラリ
`sparqlwrapper` でPythonからSPARQLを実行できる。

In [1]:
!pip install sparqlwrapper
from SPARQLWrapper import SPARQLWrapper

    100% |████████████████████████████████| 348kB 12.3MB/s 
    100% |████████████████████████████████| 51kB 20.4MB/s 


## DBpediaからSPARQLで抽出
SPARQLをインスタンス化する。 `endpoint` は叩く先のURL、 `returnFormat` は結果の形式（今回はJSON）。

In [0]:
sparql = SPARQLWrapper(endpoint='http://dbpedia.org/sparql', returnFormat='json')

クエリを書く。[このページ](http://www.aise.ics.saitama-u.ac.jp/~gotoh/IntroSPARQL.html#toc9)参照。次のセルの `.query()` で結果取得。

まずは日付型 `xsd:date` をもつプロパティを取得。

In [0]:
sparql.setQuery("select ?p where {?p a owl:DatatypeProperty ; rdfs:range xsd:date .}  group by ?p")

In [0]:
results = sparql.query().convert()

In [61]:
print(len(results["results"]["bindings"]))

145


取得した型を正規表現でリストに変換

In [58]:
import re
import joblib
properties = []
for r in results["results"]["bindings"]:
  prop = r['p']['value'].replace('http://dbpedia.org/ontology/', '')
  properties.append(prop)
joblib.dump(properties, 'properties', compress=3)

145

あるプロパティについてそのプロパティを述語としてもつトリプルを検索し、そのWikipediaページのIDを取得する。

In [0]:
sparql.setQuery("""
  PREFIX dbo: <http://dbpedia.org/ontology/>
  SELECT
    ?birth_date
    ?page
  WHERE
    { ?title dbo:birthDate ?birth_date;
    dbo:wikiPageID ?page.}
""")

In [0]:
results = sparql.query().convert()

colabから叩いてるかな？上限は10000らしい。あとでローカルでも試してみる。

In [65]:
len(results['results']['bindings'])

10000

取得した結果をデータフレームに入れる。

In [68]:
import pandas as pd
date = []
wiki_id = []
for r in results['results']['bindings']:
  date.append(r['birth_date']['value'])
  wiki_id.append(r['page']['value'])

df = pd.DataFrame(columns=['wiki_id', 'date'])
df['wiki_id'] = wiki_id
df['date'] = date
df.head()

,wiki_id,date
0,38616368,1985-1-1
1,24188476,1937-10-9
2,34851942,1896-1-1
3,35710429,1972-1-1
4,51206686,1899-1-1
